![alt text](https://github.com/callysto/callysto-sample-notebooks/blob/master/notebooks/images/Callysto_Notebook-Banner_Top_06.06.18.jpg?raw=true)  


In [ ]:
import requests as r
import pandas as pd
from pandas.io.json import json_normalize
# Assign to appropriate continent
import pycountry_convert as pc
import plotly.offline as offline 

In [ ]:
# Get the latest data
# Confirmed
API_response_confirmed = r.get("https://covid19api.herokuapp.com/confirmed")
data = API_response_confirmed.json() # Check the JSON Response Content documentation below
confirmed_df = json_normalize(data,record_path=["locations"])
# Deaths
API_response_death = r.get("https://covid19api.herokuapp.com/deaths")
data = API_response_death.json() # Check the JSON Response Content documentation below
death_df = json_normalize(data,record_path=["locations"])
# Latest
API_summary = r.get("https://covid19api.herokuapp.com/latest")
data2 = API_summary.json()
summary  = json_normalize(data2)

In [ ]:
# Get unique countries
countries_regions = confirmed_df["country"].unique()
# Get unique states
provinces_states = confirmed_df["country"].unique()

# Build country dictionary: states
country_dic  = {}
for i in range(len(countries_regions)): 

    states = confirmed_df[confirmed_df["country"]==countries_regions[i]]["province"].unique()

    country_dic[countries_regions[i]] = states

In [ ]:
# Assign continent codes for sunburst plot
cont_codes = []
for item in confirmed_df["country_code"]:
    try:
        cont_code = pc.country_alpha2_to_continent_code(item)
        cont_codes.append(cont_code)
    except:
        cont_codes.append(item)
        
confirmed_df["continent code"] = cont_codes
death_df["continent code"] = cont_codes

In [ ]:
confirmed_df.head(5)

In [ ]:
death_df.head(5)

In [ ]:
history = json_normalize(data,record_path=["locations","history"])

In [ ]:
conf_df = confirmed_df[['country','continent code','latest']]
deat_df = death_df[['country','continent code','latest']]

#The sunburst plot requires weights (values), labels, and parent (region, or World)
# We build the corresponding table here
# Inspired and adapted from https://pypi.org/project/world-bank-data/ 
columns = ['labels','parents',  'values']
# Build the levels 
# Level 1 - create copy of original 
level1 = conf_df.copy()
# Rename columns
level1.columns = columns
# Add a text column - format values column
sb = death_df.groupby('country').latest.sum().reset_index()[['country','latest']]
#level1['text'] = level1['values'].apply(lambda pop: 'Confirmed {:,.0f}'.format(pop))
#level1['text'] = sb['latest'].apply(lambda pop: 'Deaths {:,.0f}'.format(pop))
# # lat_death = sb['latest'].apply(lambda pop: 'Death {:,.0f}'.format(pop)).to_list()
# # level1['text']  = lat_death


# Create level 2
# Group by continent code
level3 = deat_df.groupby('continent code').latest.sum().reset_index()[['continent code', 'continent code', 'latest']]
# Group by continent code
level2 = conf_df.groupby('continent code').latest.sum().reset_index()[['continent code', 'continent code', 'latest']]
# Rename columns
level2.columns = columns
level2['parents'] = 'World'
# move value to text for this level
level2['text'] = level2['values'].apply(lambda pop: 'Confirmed {:,.0f}'.format(pop))
level2['values'] = level3["latest"]



# Create level 3 - world total as of latest date
level4 = pd.DataFrame({'parents': [''], 'labels': ['World'],
                       'values': [0.0], 'text': ['{:,.0f}'.format(summary["confirmed"].sum())]})

# Create master dataframe with all levels
all_levels = pd.concat([level1,level2,level4], axis=0,sort=True).reset_index(drop=True)


In [ ]:
#Sunburst plot
last_date = pd.to_datetime('today').date()
fig = offline.iplot(dict(
    data=[dict(type='sunburst', hoverinfo='values', \
               **all_levels)],
    layout=dict(title='COVID-19 Confirmed Cases and Deaths on ' + str(last_date),
                width=800, height=800,insidetextorientation='radial')),
    validate=False)

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)